In [59]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score

In [60]:
def load_data(file_path):
    data = pd.read_csv(file_path)
    data['disease'] = data.num.apply(lambda x: min(x,1))
    
    for col in data.columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    X = data.drop('disease', axis=1)
    y = data['disease']
    
    return X.values, y.values

In [61]:
X, y = load_data('data/cleveland.csv')

In [62]:
class kNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def predict(self, X):
        return np.array([self._predict(x) for x in X])

    def _predict(self, x):
        distances = [self.euclidean_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

In [63]:
def cross_validate(X, y, model, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    precisions = []
    recalls = []
    f1_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        # Print precision, recall, and f1 score for each fold
        print("Precision:", precision_score(y_test, predictions), "Recall:", recall_score(y_test, predictions), "F1:", f1_score(y_test, predictions))
        precisions.append(precision_score(y_test, predictions, zero_division=0))
        recalls.append(recall_score(y_test, predictions, zero_division=0))
        f1_scores.append(f1_score(y_test, predictions, zero_division=0))
    
    return {
        'precision': (np.mean(precisions), np.std(precisions)),
        'recall': (np.mean(recalls), np.std(recalls)),
        'f1': (np.mean(f1_scores), np.std(f1_scores))
    }

In [66]:
knn = kNN(k=3)
results = cross_validate(X, y, knn)
print(f"\nMean Results for kNN:")
print(f"Precision: {results['precision'][0]:.3f} (+/- {results['precision'][1]:.3f})")
print(f"Recall: {results['recall'][0]:.3f} (+/- {results['recall'][1]:.3f})")
print(f"F1 Score: {results['f1'][0]:.3f} (+/- {results['f1'][1]:.3f})")

Precision: 0.6875 Recall: 0.6111111111111112 F1: 0.6470588235294118
Precision: 0.6153846153846154 Recall: 0.5714285714285714 F1: 0.5925925925925926
Precision: 0.5333333333333333 Recall: 0.6153846153846154 F1: 0.5714285714285714
Precision: 0.5882352941176471 Recall: 0.7692307692307693 F1: 0.6666666666666666
Precision: 0.875 Recall: 0.4666666666666667 F1: 0.6086956521739131
Precision: 0.4444444444444444 Recall: 0.36363636363636365 F1: 0.4
Precision: 0.5 Recall: 0.5833333333333334 F1: 0.5384615384615384
Precision: 0.6153846153846154 Recall: 0.4444444444444444 F1: 0.5161290322580645
Precision: 0.8571428571428571 Recall: 0.5 F1: 0.631578947368421
Precision: 0.6153846153846154 Recall: 0.6153846153846154 F1: 0.6153846153846154

Mean Results for kNN:
Precision: 0.633 (+/- 0.133)
Recall: 0.554 (+/- 0.108)
F1 Score: 0.579 (+/- 0.074)
